# 맥주데이터 CBF 추천시스템

### 데이터 가져오기 및 정재

In [112]:
# 라이브러리
import pandas as pd
import numpy as np

from ast import literal_eval                                  # 문자열 파싱 라이브러리
from sklearn.feature_extraction.text import CountVectorizer   # 
from sklearn.metrics.pairwise import cosine_similarity        # 코사인 유사도 라이브러리

import warnings; warnings.filterwarnings('ignore')

In [113]:
# 데이터 가져오기(맥주 데이터)
beer_load = pd.read_csv('맥주이름_특징_tag_완료_최종.csv',encoding='utf-8')
rating = pd.read_csv('맥주.csv', encoding='utf-8')

In [114]:
# beer_rating: 맥주 종류별 평점평균
beer_rating = pd.DataFrame(columns=['맥주이름','평점평균','평가횟수'])
beer_rating['맥주이름'] = list(set(rating['맥주']))
beer_rating

# 평점평균 구하기
for beer in beer_rating['맥주이름']:
    temp = rating[beer==rating['맥주']]['평점'].mean()
    beer_rating['평점평균'][beer_rating['맥주이름']==beer]=temp
    
# 평가횟수 구하기
for beer in beer_rating['맥주이름']:
    temp = len(rating[beer==rating['맥주']])
    beer_rating['평가횟수'][beer_rating['맥주이름']==beer]=temp
    
beer_rating

,맥주이름,평점평균,평가횟수
0,Tiger Radler Lemon,2.07727,22
1,Sapporo Premium Beer / Draft Beer,2.275,1864
2,ARK Seoulite Ale,3.01,10
3,Berliner Kindl Jubiläums Pilsener Premium,2.61246,281
4,Leffe Brune / Bruin / Brown,3.4837,2417
...,...,...,...
72,Kronenbourg 1664 Blanc,2.78872,949
73,Stout,3.5,1
74,Jeju Wit Ale,3.00588,17
75,Warsteiner Premium Verum,2.7197,2431


In [115]:
# 총 데이터 합병
beer_data = pd.merge(beer_load, beer_rating)
beer_load.tail(50)


,Unnamed: 0,Unnamed: 0.1,맥주이름,맥주스타일,Main Category,Aroma,Flavor,Balance,Season,Paring Food,Body
27,27,16,Hite Prime Max,Pale Lager - American,라거,"['고소한 맥아향', '강한 향', '풀잎향']","['쓴 맛', '상쾌함']",['미디엄'],['연중 내내'],"['생선 요리', '해산물', '샐러드']","['드라이', '깔끔함']"
28,28,17,Hite Extra Cold,Pale Lager - American,라거,"['고소한 맥아향', '강한 향']","['쓴 맛', '상쾌함', '옅은 옥수수 향', '옅은 맥아향']","['미디엄', '단 맛']",['연중 내내'],"['생선 요리', '해산물', '샐러드']","['드라이', '깔끔함']"
29,29,21,Miller Genuine Draft (MGD),Pale Lager - American,라거,"['고소한 맥아향', '강한 향', '옥수수 향', '약한 캐라멜 멕아향']","['쓴 맛', '상쾌함', '캐러멜']",['미디엄'],['연중 내내'],"['생선 요리', '해산물', '샐러드']","['드라이', '깔끔함']"
30,30,31,Budweiser,Pale Lager - American,라거,"['고소한 맥아향', '강한 향']","['쓴 맛', '상쾌함']",['미디엄'],['연중 내내'],"['생선 요리', '해산물', '샐러드']","['드라이', '깔끔함', '가벼움', '적당한 탄산']"
31,31,41,FiLite,Pale Lager - American,라거,"['고소한 맥아향', '강한 향']","['쓴 맛', '상쾌함', '역겨움']",['미디엄'],['연중 내내'],"['생선 요리', '해산물', '샐러드']","['드라이', '깔끔함']"
32,32,45,Stephans Bräu Premium Lager,Pale Lager - American,라거,"['고소한 맥아향', '강한 향', '꽃향기', '달콤한 맥아향']","['쓴 맛', '상쾌함', '약한 단맛']",['미디엄'],['연중 내내'],"['생선 요리', '해산물', '샐러드']","['드라이', '깔끔함', '가벼움', '풍부한 탄산']"
33,33,8,Guinness Draught,Stout - Dry / Irish,영국-아일랜드 에일,"['로스팅 보리향', '커피향', '홉 아로마 거의 없음', '약한 빵 향']","['쓴 맛', '약간 크리미', '초콜렛']","['매우 드라이', '청량함']",['연중 내내'],"['스테이크', '고기 파이', '리치한 음식', '든든한 음식', '굴', '초콜릿']","['드라이', '가벼운 탄산화', '부드러움']"
34,34,9,Kronenbourg 1664 Blanc,Witbier / Belgian White Ale,컨티넨털 에일/바이스비어/에일-라거하이브리드,"['스파이시한 효모향', '오렌지', '고수', '과일향']","['드라이한 크림 풍미', '부드러움', '시큼한 끝맛', '사과맛']","['밀크셰이크 질감', '드라이', '약간 시큼함', '크리미']","['연중 내내', '따뜻한 날씨']","['홍합', '연어', '닭', '가벼운 음식']","['드라이', '미디엄', '매우 크리미', '크리미']"
35,35,43,Jeju Wit Ale,Witbier / Belgian White Ale,컨티넨털 에일/바이스비어/에일-라거하이브리드,"['스파이시한 효모향', '오렌지', '고수', '순한 고수향', '밀 향']","['드라이한 크림 풍미', '부드러움', '시큼한 끝맛']","['밀크셰이크 질감', '드라이', '약간 시큼함']","['연중 내내', '따뜻한 날씨']","['홍합', '연어', '닭', '가벼운 음식']","['드라이', '미디엄', '매우 크리미', '크리미']"
36,36,66,Platinum White Ale,Witbier / Belgian White Ale,컨티넨털 에일/바이스비어/에일-라거하이브리드,"['스파이시한 효모향', '오렌지', '고수']","['드라이한 크림 풍미', '부드러움', '시큼한 끝맛']","['밀크셰이크 질감', '드라이', '약간 시큼함']","['연중 내내', '따뜻한 날씨']","['홍합', '연어', '닭', '가벼운 음식']","['드라이', '미디엄', '매우 크리미', '크리미']"


In [116]:
# 데이터 준비 및 정재 list 형태로 바꿔주기 
# beer_data에서 총 6개의 feature(Aroma, Flavor, Balance, Season, Paring  Food, Body)를 가져온다 
beer_df = pd.DataFrame(beer_data, columns = ['Aroma','Flavor','Balance','Season','Paring Food', 'Body'])


# 데이터 타입 확인: data셀 하나에 들어있는 내용들은 str
type(beer_df['Aroma'].iloc[0])
beer_df['Aroma'].iloc[0]


# str형태에서 list로 변환
beer_df['Aroma'] = beer_df['Aroma'].apply(literal_eval)
beer_df['Flavor'] = beer_df['Flavor'].apply(literal_eval)
beer_df['Balance'] = beer_df['Balance'].apply(literal_eval)
beer_df['Season'] = beer_df['Season'].apply(literal_eval)
beer_df['Paring Food'] = beer_df['Paring Food'].apply(literal_eval)
beer_df['Body'] = beer_df['Body'].apply(literal_eval)

In [117]:
beer_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77 entries, 0 to 76
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Aroma        77 non-null     object
 1   Flavor       77 non-null     object
 2   Balance      77 non-null     object
 3   Season       77 non-null     object
 4   Paring Food  77 non-null     object
 5   Body         77 non-null     object
dtypes: object(6)
memory usage: 4.2+ KB


In [118]:
beer_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77 entries, 0 to 76
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     77 non-null     int64 
 1   Unnamed: 0.1   77 non-null     int64 
 2   맥주이름           77 non-null     object
 3   맥주스타일          77 non-null     object
 4   Main Category  77 non-null     object
 5   Aroma          77 non-null     object
 6   Flavor         77 non-null     object
 7   Balance        77 non-null     object
 8   Season         77 non-null     object
 9   Paring Food    77 non-null     object
 10  Body           77 non-null     object
 11  평점평균           77 non-null     object
 12  평가횟수           77 non-null     object
dtypes: int64(2), object(11)
memory usage: 8.4+ KB


## 데이터 이름 전처리 - Leffe Brune / ... -> Leffe Brune

In [119]:
ogname = ['Sapporo Premium Beer / Draft Beer',
         'Guinness Original 4.2% (Ireland/UK)',
         'Franziskaner Hefe-Weissbier / Weissbier Naturtrub',
         'Bavaria Pilsener / Premium Beer',
         'Leffe Brune / Bruin / Brown']
name = [
    'Sapporo Premium Beer',
    'Guinness Original 4.2%',
    'Franziskaner Hefe-Weissbier',
    'Bavaria Pilsener',
    'Leffe Brune'
]

In [120]:
# 전처리 전
beer_data[beer_data['맥주이름'].str.contains('/')]['맥주이름'].unique()

array(['Sapporo Premium Beer / Draft Beer',
       'Bavaria Pilsener / Premium Beer',
       'Franziskaner Hefe-Weissbier / Weissbier Naturtrub',
       'Guinness Original 4.2% (Ireland/UK)',
       'Leffe Brune / Bruin / Brown'], dtype=object)

In [121]:
for i in range(len(ogname)):
    beer_data['맥주이름'] = beer_data['맥주이름'].apply(lambda x : name[i] if x==ogname[i] else x)

In [122]:
# 전처리 후
beer_data[beer_data['맥주이름'].str.contains('/')]['맥주이름'].unique()

array([], dtype=object)

In [123]:
beer_data.to_csv('맥주_cbf_data.csv', encoding='utf-8')
beer_df.to_csv('맥주_cbf_feature.csv', encoding='utf-8')

# 장르 컨텐츠 유사도 측정(아래는 1개의 항목에 대해서 각각 실행해 본 것

### Something에 무엇을 넣느냐에 따라 달라집니다.

### (총 5개의 항목 'Aroma','Flavor','Balance','Paring Food', 'Body')

In [124]:
# something = 'Aroma'
# something = 'Flavor'
# something = 'Balance'
# something = 'Paring Food'
something = 'Body'

In [125]:
# CountVectorizer를 활용하기 위한 전처리 
beer_df[something +'_literal'] = beer_df[something].apply(lambda x : (' ').join(x))
beer_df[something +'_literal']

#CountVectorizer로 학습시키기
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))   # 특징 2개(ngram_range: 1<= n <=2)
something_mat = count_vect.fit_transform(beer_df[something + '_literal'])
print(something_mat.shape)
# 77개의 맥주 중에 178개 Aroma의 'Aroma 매트릭스'가 생성됨

(77, 71)


### 코사인 유사도(cosine_similarity) 이용해서 영화별 유사도 계산

In [98]:
# 77개의 맥주에 대한 가각 유사한 맥주들이 계산 됨

something_sim = cosine_similarity(something_mat,something_mat)
print(something_sim.shape)
print(something_sim)


(77, 77)
[[1.         0.62994079 0.75592895 ... 0.37796447 0.21821789 0.12598816]
 [0.62994079 1.         0.55555556 ... 0.33333333 0.19245009 0.11111111]
 [0.75592895 0.55555556 1.         ... 0.33333333 0.19245009 0.11111111]
 ...
 [0.37796447 0.33333333 0.33333333 ... 1.         0.         0.        ]
 [0.21821789 0.19245009 0.19245009 ... 0.         1.         0.19245009]
 [0.12598816 0.11111111 0.11111111 ... 0.         0.19245009 1.        ]]


In [99]:
# 순서 보기
# 숫자가 작을수록 유사도가 높은 맥주
something_sim_sorted_ind = something_sim.argsort()[:, ::-1]
print(something_sim_sorted_ind[:1])

[[ 0 21 18 15 14 10  8 30 23  9 32  2  6  3 25 20 31 29 28 27 26 24 16 17
   5  1 22  4 12 11  7 50 69 19 37 13 38 58 57 56 54 52 74 73 49 55 40 59
  46 43 63 75 53 47 48 62 45 44 64 61 51 41 42 33 76 34 35 36 60 65 66 67
  68 70 71 72 39]]


## 데이터 이름 전처리 - Leffe Brune / ... -> Leffe Brune

In [100]:
ogname = ['Sapporo Premium Beer / Draft Beer',
         'Guinness Original 4.2% (Ireland/UK)',
         'Franziskaner Hefe-Weissbier / Weissbier Naturtrub',
         'Bavaria Pilsener / Premium Beer',
         'Leffe Brune / Bruin / Brown']
name = [
    'Sapporo Premium Beer',
    'Guinness Original 4.2%',
    'Franziskaner Hefe-Weissbier',
    'Bavaria Pilsener',
    'Leffe Brune'
]

In [101]:
# 전처리 전
beer_data[beer_data['맥주이름'].str.contains('/')]['맥주이름'].unique()

array([], dtype=object)

In [102]:
for i in range(len(ogname)):
    beer_data['맥주이름'] = beer_data['맥주이름'].apply(lambda x : name[i] if x==ogname[i] else x)

In [103]:
# 전처리 후
beer_data[beer_data['맥주이름'].str.contains('/')]['맥주이름'].unique()

array([], dtype=object)

In [104]:
beer_data.to_csv('맥주_cbf_data.csv', encoding='utf-8')

#### 성향이 비슷한 맥주 10개 추천

In [105]:
def find_sim_beer_ver1(beer_data, sorted_ind, beer_name, top_n=10):
    
    beer_name = beer_data[beer_data['맥주이름'] == beer_name]
    
    beer_index = beer_name.index.values
    similar_indexes = sorted_ind[beer_index, :(top_n)]
    
    print(similar_indexes)
    
    similar_inidexes = similar_indexes.reshape(-1)
    
    return beer_data.iloc[similar_inidexes]

In [106]:
beer_name = 'Leffe Brune'

In [110]:
# 10개의 맥주 추천 결과
# Pilsner Urquel로 검색한 결과, 대체로 필스터 혹은 비슷한 성질으 가진 페일에일이 추천되는 것을 알 수 있다.

similar_beers = find_sim_beer_ver1(beer_data, something_sim_sorted_ind, beer_name,3)
similar_beers

[[70 39 68]]


,Unnamed: 0,Unnamed: 0.1,맥주이름,맥주스타일,Main Category,Aroma,Flavor,Balance,Season,Paring Food,Body,평점평균,평가횟수
70,70,60,Leffe Brune,Belgian Ale - Dark / Amber,미국-크래프트 맥주,"['약한 빵 향', '과일향', '스파이시한 향', '카라멜 향']","['섬세한 맥아 풍미', '약한 캐러멜', '깔끔한 쓴맛', '단맛']","['균형', '균등', '청량함', '드라이한 끝맛', '부드러움']",['연중 내내'],"['라이트한 치즈', '홍합', '해산물 요리', '광범위한 음식']",['미디엄'],3.4837,2417
39,39,14,Stout,Porter,영국-아일랜드 에일,"['맥아 로스팅향', '홉 아로마 거의 없음', '초콜렛 향', '커피향']","['로스팅과 구운 풍미', '크리미한 맥아 풍미', '홉 풍미 거의 없음', '커피...","['다양한 밸런스를 가진 제품 존재', '다양한 정도의 로스팅 풍미', '크리미']","['연중 내내', '서늘한 계절']","['바베큐', '소시지', '초콜릿칩 쿠키', '구운 음식', '훈제된 음식']",['미디엄'],3.5,1
68,68,50,Egger Märzenbier,Märzen / Oktoberfest Bier,라거,"['고소한 맥아향', '캐러멜향', '쿠키향', '홉 아로마 거의 없음']","['캐러멜', '달콤한 쿠기 풍미', '약간 쌉쌀함']","['맥아 풍미', '홉에 의한 밸런스 거의 없음']","['9월', '10월', '연중내내']","['멕시코 요리', '매운 요리', '닭', '소시지', '마일드 치즈']",['미디엄'],2.5,183


In [127]:
beer_load = pd.read_csv('맥주이름_특징_tag_완료_최종.csv', encoding='utf-8')
rating = pd.read_csv('맥주.csv', encoding='utf-8')

# beer_rating: 맥주 종류별 평점평균
beer_rating = pd.DataFrame(columns=['맥주이름', '평점평균', '평가횟수'])
beer_rating['맥주이름'] = list(set(rating['맥주']))

# 평점평균 구하기
for beer in beer_rating['맥주이름']:
    temp = rating[beer == rating['맥주']]['평점'].mean()
    beer_rating['평점평균'][beer_rating['맥주이름'] == beer] = temp

# 평가횟수 구하기
for beer in beer_rating['맥주이름']:
    temp = len(rating[beer == rating['맥주']])
    beer_rating['평가횟수'][beer_rating['맥주이름'] == beer] = temp

# 총 데이터 합병
beer_data = pd.merge(beer_load, beer_rating, on='맥주이름')

# 데이터 준비 및 정재 list 형태로 바꿔주기
# beer_data에서 총 6개의 feature(Aroma, Flavor, Balance, Season, Paring  Food, Body)를 가져온다
beer_df = pd.DataFrame(beer_data, columns=['Aroma', 'Flavor', 'Balance', 'Season', 'Paring Food', 'Body'])

# str형태에서 list로 변환
beer_df['Aroma'] = beer_df['Aroma'].apply(literal_eval)
beer_df['Flavor'] = beer_df['Flavor'].apply(literal_eval)
beer_df['Balance'] = beer_df['Balance'].apply(literal_eval)
beer_df['Season'] = beer_df['Season'].apply(literal_eval)
beer_df['Paring Food'] = beer_df['Paring Food'].apply(literal_eval)
beer_df['Body'] = beer_df['Body'].apply(literal_eval)

ogname = ['Sapporo Premium Beer / Draft Beer',
          'Guinness Original 4.2% (Ireland/UK)',
          'Franziskaner Hefe-Weissbier / Weissbier Naturtrub',
          'Bavaria Pilsener / Premium Beer',
          'Leffe Brune / Bruin / Brown']
name = [
    'Sapporo Premium Beer',
    'Guinness Original 4.2%',
    'Franziskaner Hefe-Weissbier',
    'Bavaria Pilsener',
    'Leffe Brune'
]

for i in range(len(ogname)):
    beer_data['맥주이름'] = beer_data['맥주이름'].apply(lambda x: name[i] if x == ogname[i] else x)

# 맥주 이름, 사용자 취향 받아오기
beer_name = 'Cass Fresh'
detail = 'Body'

something = detail

# str형태에서 list로 변환
# beer_df['Aroma'] = beer_df['Aroma'].apply(literal_eval)
# beer_df['Flavor'] = beer_df['Flavor'].apply(literal_eval)
# beer_df['Balance'] = beer_df['Balance'].apply(literal_eval)
# beer_df['Season'] = beer_df['Season'].apply(literal_eval)
# beer_df['Paring Food'] = beer_df['Paring Food'].apply(literal_eval)
# beer_df['Body'] = beer_df['Body'].apply(literal_eval)

def find_sim_beer_ver1(beer_data, sorted_ind, beer_name, top_n=10):

    beer_name = beer_data[beer_data['맥주이름'] == beer_name]

    beer_index = beer_name.index.values
    similar_indexes = sorted_ind[beer_index, :(top_n)]

    print(similar_indexes)

    similar_inidexes = similar_indexes.reshape(-1)

    return beer_data.iloc[similar_inidexes]

# CountVectorizer를 활용하기 위한 전처리
beer_df[something + '_literal'] = beer_df[something].apply(lambda x: (' ').join(x))

# CountVectorizer로 학습시키기
count_vect = CountVectorizer(min_df=0, ngram_range=(1, 2))  # 특징 2개(ngram_range: 1<= n <=2)
something_mat = count_vect.fit_transform(beer_df[something + '_literal'])

# 77개의 맥주에 대한 가각 유사한 맥주들이 계산 됨
something_sim = cosine_similarity(something_mat, something_mat)

# 순서 보기
# 숫자가 작을수록 유사도가 높은 맥주
something_sim_sorted_ind = something_sim.argsort()[:, ::-1]

similar_beers = find_sim_beer_ver1(beer_data, something_sim_sorted_ind, beer_name, 4)

similar_beers = similar_beers[similar_beers['맥주이름'] != beer_name]

[[30  6  8  0]]


In [129]:
tmp = pd.read_csv('clustering_전체결과.csv', encoding='utf-8')

In [161]:
ttmp = pd.read_csv('맥주이름.csv', encoding='utf-8')

In [172]:
sujin = pd.read_csv('맥주이름_특징_tag_완료_최종.csv', encoding='utf-8', index_col=0)

In [173]:
sujin.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77 entries, 0 to 76
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0.1   77 non-null     int64 
 1   맥주이름           77 non-null     object
 2   맥주스타일          77 non-null     object
 3   Main Category  77 non-null     object
 4   Aroma          77 non-null     object
 5   Flavor         77 non-null     object
 6   Balance        77 non-null     object
 7   Season         77 non-null     object
 8   Paring Food    77 non-null     object
 9   Body           77 non-null     object
dtypes: int64(1), object(9)
memory usage: 6.6+ KB


In [174]:
tmp.sort_values(by='맥주').맥주.unique()

array(['2M', '5,0 Original Pils', 'Aass Bock', 'Abbaye des Rocs Brune',
       'Afsana', 'Ailyak Cryo Mosaic IPA', 'Albani Giraf Beer',
       'Albrau Premium Pils', 'Aldaris Porteris', 'Alfa [Αλφα]',
       'Alhambra Reserva 1925', 'Alivaria Porter', 'Almaza Pilsener',
       "Almus T'mno (Dark)", 'Alpine Beer Company Pure Hoppiness',
       'Amadeus Biere Blanche', 'Amager Hr. Papsø', 'Ambar Especial',
       'Amsterdam Boneshaker IPA', 'Anchor Steam Beer', 'Andes',
       'Anker Bir Pilsener', 'Antarctica Pilsen', 'Antares Barley Wine',
       'Apatinsko Svetlo Pivo', 'Ararat Beer', 'Arboga 10.2%',
       'Arcobräu Weissbier Hell', 'Arctic Fox Ice Cap Lager',
       'Arequipeña Rubia (Blond)', 'Argo Lager Beer', 'Ariana', 'Aris',
       'Arpa', 'Asahi Super Dry', 'Asahi Super Dry Black', 'Asbeer',
       'Asmara Lager Beer', 'Astika Fine Quality Lager',
       'Atlas Three Sisters (Bottle)', 'Augustiner Heller Bock',
       'Austral Lager', 'Axt',
       'B.B. Originál Světlý Ležák 

In [175]:
name = ttmp.맥주.unique().tolist()

In [176]:
sujin['맥주이름'] = sujin['맥주이름'].apply(lambda x: x if x in name else '삭제')

In [177]:
ogdata = sujin

In [178]:
ogdata = ogdata[ogdata['맥주이름'] != '삭제']

In [179]:
ogdata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60 entries, 0 to 75
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0.1   60 non-null     int64 
 1   맥주이름           60 non-null     object
 2   맥주스타일          60 non-null     object
 3   Main Category  60 non-null     object
 4   Aroma          60 non-null     object
 5   Flavor         60 non-null     object
 6   Balance        60 non-null     object
 7   Season         60 non-null     object
 8   Paring Food    60 non-null     object
 9   Body           60 non-null     object
dtypes: int64(1), object(9)
memory usage: 5.2+ KB


In [180]:
ogdata.to_csv('맥주이름_특징_tag_완료_최종.csv', encoding='utf-8')